# Introduction to AVISO-data

We will use the satellite-altimetry-derived ocean-surface current product AVISO-data as a dataset to validate the methods. The data is freely available from the [AVISO]((http://aviso.oceanobs.com)).

The sea surface height $ h $ acts as a stream function for the geostrophic ocean surface velocity field. Particle trajectories on the ocean surface are approximately trajectories of the 2D system of ODEs:

\begin{align}
\dot{\varphi} &= -\dfrac{g}{R^2f(\theta)\cos(\theta)}\partial_{\theta}h(\phi, \theta, t) \\
\dot{\theta} &= \dfrac{g}{R^2f(\theta)\cos(\theta)}\partial_{\varphi}h(\phi, \theta, t),
\end{align}

where $ \varphi, \theta $ respectively are the longitudinal and latitudinal position of the particle, $ g $ is the constant of gravity, $ R $ is the mean radius, $ f(\theta) = 2\Omega \sin(\theta) $ is the coriolis parameter and $ \Omega $ is the mean angular velocity of the earth.

The AVISO-data provides global coverage of ocean currents at weekly time-intervals with a spatial resolution of $ 0.25^{\circ} $. As part of this work, we focus on the commonly studied Agulhas region in the Southern Ocean:

\begin{equation}
(\varphi, \theta) \in [14^{\circ}W, 9^{\circ}E] \times [39^{\circ}S, 21^{\circ}S]
\end{equation}

over a period of (maximum) 90 days starting at $ t_0 = 24 $ November 2006. The area under study is characterized by intense eddy activity.